In [1]:
import jax.numpy as jnp
import jax
from whisper_jax import FlaxWhisperForConditionalGeneration
from functions import *
from functools import partial

In [2]:
speech_path = 'data/train_mp3s/'
pad_to_multiple_of = 8
batch_size = 4
dtype = jnp.float16

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v2", language="bn", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v2")
text = pd.read_csv('data/train.csv')

In [3]:
dataset = AudioDataset(text,speech_path)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=8, \
                        collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,pad_to_multiple_of=pad_to_multiple_of))

In [4]:
out = next(iter(dataloader))
audio,input_ids,attention_mask = map(partial(jnp.array,dtype=dtype),out)

In [6]:
# load the processor and model
model, params = FlaxWhisperForConditionalGeneration.from_pretrained(
    "openai/whisper-large-v2", dtype=dtype, _do_init=False,)

In [7]:
audio.shape,attention_mask.shape,input_ids.shape

((4, 80, 320), (4, 128), (4, 128))

In [8]:
out = model(audio,input_ids,decoder_attention_mask=attention_mask,params=params,train=True)

In [12]:
out.logits.shape

(4, 128, 51865)

In [13]:
type(out)

transformers.modeling_flax_outputs.FlaxSeq2SeqLMOutput